# Лабораторная работа №2: Reports with Apache Spark.<br>Выполнил студент группы 6133-010402D Мелешенко Иван.

# Задание

##[Текст задания лабораторной работы](https://gitlab.com/ssau.tk.courses/big_data/-/tree/master/L2%20-%20Reports%20with%20Apache%20Spark)

1. Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.
2. Получившийся отчёт сохранить в формате Apache Parquet.

# Этап 1. Подготовка средств и данных.

## Шаг 1. Установка PySpark

PySpark — это API Apache Spark, применяемый для распределенной обработки больших данных.

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 62.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=e23c5868ecbbc39f7405a5df5b0bb7c0021ab79c7b83e305e2388268de0cc67b
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


## Шаг 2. Импорт необходимых библиотек.

In [2]:
import pyspark
import os

from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

## Шаг 3. Создание проекта.

### Создание точки конфигурации, установка имени проекта, и выбора подключения к кластеру


In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
sc = SparkSession.builder.appName("L2ApacheReports").master("local[*]").getOrCreate()

### Проверка установки

In [4]:
sc

## Шаг 4. Загрузка и обработка данных

### Подключение гугл диска

Для загрузки файлов на гугл коллаб воспользуемся гугл диском, который максимально ускорит процесс загрузки.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Загрузка файла с языками программирования

In [6]:
listProgLanguages = sc.read.csv("/content/drive/MyDrive/programming-languages.csv")

### Создание списка языков программиорвания

In [7]:
languageList = [str(x[0]) for x in listProgLanguages.collect()]

### Просмотр созданного списка

Выведем первые 10 языков программирования

In [9]:
languageList[:10]

['name',
 'A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET']

### Корректировка спиская языков программирования

Удалим первую ячейку языкового списка, поскольку она содержит заголовок файла, и не нужна.

In [10]:
languageList.pop(0)

'name'

### Просмотр списка языков программирования

In [11]:
languageList[:10]

['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS']

### Загрузка файла с выборкой данных по использованию языков программирования

In [12]:
postsSampleData = sc.read.format("xml").options(rowTag="row").load('/content/drive/MyDrive/posts_sample.xml')

In [18]:
postsSampleData.take(3)

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUs

# Этап 2. Описание функций для обработки текста.

## Функция определения языка программирования

Функция определения языка программирования производит перевод всего текста в нижний регистр и ищет название языка программирования в каждой строке.

В случае нахождения язык порграммирования создается кортеж, иначе возвращается None

In [19]:
def detectionProgLanguage(x):
  tag = None
  for language in languageList:
    if "<" + language.lower() + ">" in x._Tags.lower():
      tag = language
      break
  if tag is None:
    return None
  return (x._Id, tag)

## Функция проверки даты

Данная функция производит фильтрацию информации по датам, так как необходима информация с 2010 по 2020 год.

In [20]:
def detectionYear(x, year):
  start = datetime(year=year, month=1, day=1)
  end = datetime(year=year, month=12, day=31)
  CreationDate = x._CreationDate
  return CreationDate >= start and CreationDate <= end

# Этап 3. Выполнение заданий

## Задание 1. Формирование отчетов

In [21]:
selectionProgLanguage = {}
for year in range(2010, 2020):
  selectionProgLanguage[year] = postsSampleData.rdd\
      .filter(lambda x: x._Tags is not None and detectionYear(x, year))\
      .map(detectionProgLanguage)\
      .filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1])\
      .aggregateByKey(
          0,
          lambda x, y: x + 1,
          lambda x1, x2: x1 + x2,
      )\
      .sortBy(lambda x: x[1], ascending=False)\
      .toDF()
  selectionProgLanguage[year] = selectionProgLanguage[year].select(col("_1").alias("Programming_language"), 
                                                 col("_2").alias(f"Number_of_mentions_in_{year}")).limit(10)

## <center> ***Ответ на задание 1: Вывод отчетов***</center>

In [23]:
for year in range(2010, 2020):
  selectionProgLanguage[year].show()

+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2010|
+--------------------+--------------------------+
|                Java|                        52|
|          JavaScript|                        44|
|                 PHP|                        42|
|              Python|                        25|
|         Objective-C|                        22|
|                   C|                        20|
|                Ruby|                        11|
|              Delphi|                         7|
|                   R|                         3|
|                Bash|                         3|
+--------------------+--------------------------+

+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2011|
+--------------------+--------------------------+
|                 PHP|                        97|
|                Java|                        92|
|          JavaScript|                        82|

## Задание 2. Сохрание отчетов

In [29]:
for year in selectionProgLanguage.keys():
    selectionProgLanguage[year].write.format("parquet").save(f"/content/drive/MyDrive/top_lang/top_{year}")